# Aula 1 - Consultando o banco de dados com um assistente

Usamos a biblioteca [sqlalchemy](https://docs.sqlalchemy.org/en/20/) para trabalhar com bancos de dados no ambiente Python.
- O [`create_engine`](https://docs.sqlalchemy.org/en/20/core/engines.html) cria a conexão com o banco de dados usando a URL fornecida.
- O [`MetaData`](https://docs.sqlalchemy.org/en/20/core/metadata.html) gerencia informações sobre as tabelas e seus esquemas (metadados).

* [`pd.read_sql_table`](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_table.html) carrega a tabela diretamente para um DataFrame do pandas

## Configurando o Llama Index

In [ ]:
!pip install llama-index llama-index-llms-groq llama-index-experimental llama-index-embeddings-huggingface llama-index-postprocessor-cohere-rerank gradio

In [ ]:
from google.colab import userdata
key = userdata.get('GROQ_API')

Informações sobre os modelos:
- [`llama-3.1-70b-versatile`](https://github.com/meta-llama/llama-models/blob/main/models/llama3_1/MODEL_CARD.md)
- [`BAAI/bge-m3`](https://huggingface.co/BAAI/bge-m3)

In [ ]:
modelo="llama-3.1-70b-versatile"
modelo_hf_emb="BAAI/bge-m3"

- `SQLDatabase` conecta ao banco de dados SQL.
- `SQLTableNodeMapping` cria um mapeamento do banco de dados.

* `SQLTableSchema` representa cada tabela no banco como um objeto e podem conter informações sobre os nomes das tabelas e, eventualmente, descrições.

* `ObjectIndex` combina os objetos de esquema (SQLTableSchema) e o mapeamento (SQLTableNodeMapping) para construir um índice vetorial com embeddings.

* `retriever` é um objeto usado para encontrar a tabela ou objeto mais relevante no banco.

## Consultando o banco de dados

* `SQLTableRetrieverQueryEngine` interpreta e executa consultas SQL com base em entrada de linguagem natural.

1° consulta

```
'Quais estados mais frequentes na tabela Clientes?'
```

2° consulta

```
'Quais as fornecedoras localizadas na cidade de São Paulo?'
```

3° consulta

```
'Quantos funcionários com o cargo de vendedores temos cadastrados?'
```

# Aula 2 - Contextualizando o modelo

Utilizando LLM do Groq com LlamaIndex
- [Guia Groq](https://docs.llamaindex.ai/en/stable/examples/llm/groq/)

In [ ]:
def gerar_descricao_tabela(nome_tabela, df_amostra):
    prompt = f"""
    Analise a amostra da tabela '{nome_tabela}' abaixo e forneça uma curta e breve descrição do conteúdo dessa tabela.
    Informe até o máximo de 5 valores únicos de cada coluna.


    Amostra da Tabela:
    {df_amostra}

    Descrição:
    """

    resposta = llm.complete(prompt = prompt)

    return resposta.text

A descrição das tabelas precisa ser adicionada no schema das tabelas

1° consulta com nova `query_engine`

```
'Quais as fornecedoras localizadas na cidade de São Paulo?'
```

2° consulta com nova `query_engine`

```
'Quantos funcionários com o cargo de vendedores temos cadastrados?'
```

# Aula 3 - Iniciando a personalização de prompts

**Estrutura da cadeia passos**

![Alt text: O diagrama inicia com uma "Entrada", que leva a várias etapas sequenciais e decisionais.
Depois da entrada, segue-se uma etapa chamada "Acesso à tabela", após ela a etapa "Contexto da tabela" e depois para o "1º Prompt". Estas etapas parecem preparar a consulta inicial, contextualizando-a antes de processá-la através de um modelo de linguagem de aprendizado profundo (LLM), indicado no fluxo.
Após o LLM processar o primeiro prompt, os resultados são utilizados para obter uma "Consulta SQL" e, a partir dela, o "Resultado Consulta SQL". O fluxo prossegue para um "2º Prompt (Síntese)", sugerindo uma etapa de elaboração ou síntese baseada no resultado e a consulta SQL.
Finalmente, a saída deste segundo prompt aplicando em uma LLM leva à "Resposta final", indicando a conclusão do processo.](https://github.com/Mirlaa/llamaindex-assistente-BD/blob/main/Suporte-educacional/esquema-Assitente-BD-Zoop.png?raw=true)

## Criação do 1° Prompt

In [ ]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
print(DEFAULT_TEXT_TO_SQL_PROMPT.template)

In [ ]:
texto2sql = """Dada uma pergunta em linguagem natural, crie uma consulta {dialect} sintaticamente correta para executar e, em seguida, verifique os resultados da consulta e retorne a resposta. Você pode ordenar os resultados por uma coluna relevante para retornar os exemplos mais informativos no banco de dados.

Nunca consulte todas as colunas de uma tabela específica. Pergunte apenas por algumas colunas relevantes, de acordo com a pergunta.

Preste atenção para usar apenas os nomes de colunas que você pode ver na descrição do esquema. Tenha cuidado para não consultar colunas que não existem. Preste atenção em qual coluna está em qual tabela. Além disso, qualifique os nomes das colunas com o nome da tabela quando necessário.

Use o seguinte formato, cada um em uma linha:

Pergunta: Pergunta aqui
ConsultaSQL: Consulta SQL para executar
ResultadoSQL: Resultado da ConsultaSQL
Resposta: Resposta final aqui

Use apenas as tabelas listadas abaixo.

{schema}

Pergunta: {pergunta_user}
ConsultaSQL:
"""

* `PromptTemplate` permite criar um template de um prompt para ser enviado à uma LLM.

## Descrição tabelas

* `FnComponent` é uma classe do LlamaIndex usada para encapsular funções como componentes reutilizáveis dentro de uma Query Pipeline.

## Consulta SQL e resultado

* `SQLRetriever` executa consultas SQL diretamente no banco de dados configurado.

# Aula 4 - Construindo a cadeia de ações

**CRIAÇÃO DO 2° PROMPT**

In [ ]:
prompt_2_str = '''
    Você é o "Assitente de consulta de banco de dados da Zoop".
    Dada a seguinte pergunta, a consulta SQL correspondente e o resultado SQL, responda à pergunta de modo agradável e objetivamente.
    Evite iniciar conversas com cumprimentos e apresentações, como "Olá".

    Pergunta: {pergunta_user}
    Consulta SQL: {consulta}
    Resultado SQL: {resultado}
    Resposta:
    '''

prompt_2 = PromptTemplate(
    prompt_2_str,
)

**DEFINIÇÃO DOS MÓDULOS DA ESTRUTURA**

**Vamos usar nossa estrutura de apoio:**

![Alt text: O diagrama inicia com uma "Entrada", que leva a várias etapas sequenciais e decisionais.
Depois da entrada, segue-se uma etapa chamada "Acesso à tabela", após ela a etapa "Contexto da tabela" e depois para o "1º Prompt". Estas etapas parecem preparar a consulta inicial, contextualizando-a antes de processá-la através de um modelo de linguagem de aprendizado profundo (LLM), indicado no fluxo.
Após o LLM processar o primeiro prompt, os resultados são utilizados para obter uma "Consulta SQL" e, a partir dela, o "Resultado Consulta SQL". O fluxo prossegue para um "2º Prompt (Síntese)", sugerindo uma etapa de elaboração ou síntese baseada no resultado e a consulta SQL.
Finalmente, a saída deste segundo prompt aplicando em uma LLM leva à "Resposta final", indicando a conclusão do processo.](https://github.com/Mirlaa/llamaindex-assistente-BD/blob/main/Suporte-educacional/esquema-Assitente-BD-Zoop.png?raw=true)

## Definindo conexão de ações

## Testando as saídas

1° consulta

```
'Quais estados mais frequentes na tabela Clientes?'
```

2° consulta

```
'Quantos funcionários com o cargo de vendedores temos cadastrados?'
```

# Aula 5 - Criando um chat com a IA

**Função para executar uma consulta**

**Função para adicionar a resposta da IA ao histórico de conversa**

**Iniciar a construção da interface**

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown('## Chat com Assistente SQL')
    gr.Markdown(
        '''
        Este é um assistente SQL interativo, projetado para responder perguntas sobre os dados da loja Zoop.
        Insira sua pergunta no campo abaixo e o assistente irá responder com base no resultado da consulta SQL
        nos dados disponíveis.
        '''
    )


Podemos publicar nosso projeto no [Hugging Face](https://huggingface.co/new-space) e divulgar nosso trabalho!

> O projeto que você construiu é seu! Então fique a vontade para compartilhar o seu projeto no GitHub, Hugging Face, expor no seu portfólio ou divulgar no LinkedIn e nos marcar na postagem!
>
> Bons estudos!

--Mirla Costa